In [18]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
from multiprocessing import Process, Pipe
import time
import lightgbm as lgb
from sklearn.model_selection import KFold
# Any results you write to the current directory are saved as output.
%matplotlib inline

['mulliken_charges.csv', 'test.csv', 'structures.csv', 'dipole_moments.csv', 'potential_energy.csv', 'train.csv', 'scalar_coupling_contributions.csv', 'magnetic_shielding_tensors.csv', 'sample_submission.csv']


In [2]:
jobs = 10

In [3]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')
structure_df = pd.read_csv('../input/structures.csv')

In [4]:
euclidean_distance = pd.read_pickle('distance.pkl')
h_num = pd.read_pickle('h_num.pkl')

In [5]:
train_df = pd.concat([train_df, pd.DataFrame(data=euclidean_distance, columns=['euclidean_distance']),
                                pd.DataFrame(data=h_num, columns=['h_num'])], axis=1)

In [6]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,euclidean_distance,h_num
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1.091953,4.0
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,1.783120,4.0
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,1.783147,4.0
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,1.783157,4.0
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1.091952,4.0


In [8]:
y = train_df['scalar_coupling_constant']
X = train_df[['type','euclidean_distance']]

In [11]:
X.dtypes

type                   object
euclidean_distance    float64
dtype: object

In [12]:
X = pd.get_dummies(X)

In [13]:
X.head()

,euclidean_distance,type_1JHC,type_1JHN,type_2JHC,type_2JHH,type_2JHN,type_3JHC,type_3JHH,type_3JHN
0,1.091953,1,0,0,0,0,0,0,0
1,1.783120,0,0,0,1,0,0,0,0
2,1.783147,0,0,0,1,0,0,0,0
3,1.783157,0,0,0,1,0,0,0,0
4,1.091952,1,0,0,0,0,0,0,0


In [14]:
X.shape, y.shape

((4658147, 9), (4658147,))

In [20]:
X = X.values

In [36]:
y = y.values

In [25]:
train_df['type'].values

array(['1JHC', '2JHH', '2JHH', ..., '3JHC', '2JHC', '1JHC'], dtype=object)

In [39]:
def Evaluation(y, pred):
    types = train_df['type'].values
    dic = {'1JHC':[], '2JHH':[], '1JHN':[], '2JHN':[], '2JHC':[], '3JHH':[], '3JHC':[], '3JHN':[]}
    loss = np.abs(y - pred)
    score=0
    for i in range(len(y)):
        dic[types[i]].append(loss[i])
    for i in dic.keys():
        score+=np.log(np.sum(dic[i])/len(dic[i]))
    score/=len(dic)
    return score

In [38]:
kf = KFold(n_splits=10)
score = []
for train_idx, test_idx in tqdm(kf.split(X), total=10):
    train_X, test_X = X[train_idx], X[test_idx]
    train_y, test_y = y[train_idx], y[test_idx]

    #model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    model = lgb.LGBMModel(objective='regression')
    model.fit(train_X, train_y)
    pred = model.predict(test_X)
    score.append(Evaluation(test_y, pred))
print(score)
print(np.mean(score))







  0%|          | 0/10 [00:00<?, ?it/s]





 10%|█         | 1/10 [00:05<00:45,  5.03s/it]





 20%|██        | 2/10 [00:10<00:40,  5.05s/it]





 30%|███       | 3/10 [00:15<00:35,  5.05s/it]





 40%|████      | 4/10 [00:19<00:29,  4.97s/it]





 50%|█████     | 5/10 [00:24<00:24,  4.96s/it]





 60%|██████    | 6/10 [00:30<00:20,  5.10s/it]





 70%|███████   | 7/10 [00:35<00:15,  5.23s/it]





 80%|████████  | 8/10 [00:41<00:10,  5.21s/it]





 90%|█████████ | 9/10 [00:46<00:05,  5.18s/it]





100%|██████████| 10/10 [00:51<00:00,  5.30s/it]







[0.8316957471470484, 1.1906631373937309, 1.0466684416552297, 1.0920367677803842, 0.9327248184872994, 0.9361804438376584, 0.918821961824771, 0.9039362462707173, 0.8801650854723104, 0.9961407695100697]
0.9729033419379218
